<a href="https://colab.research.google.com/github/jjang750/fullstack-gpt/blob/master/RAG_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformes.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.1 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
model_id = "kyujinpy/Ko-PlatYi-6B" # model id

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

print(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
  

In [10]:
device = "cuda:0"

message = [
    {"role":"user", "content":"은행의 기준 금리에 대해서 설명해줘?"}
]

encodeds = tokenizer.apply_chat_template(message,  return_tensors="pt") # use pytorch

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)

print(decoded[0])


<|startoftext|> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

은행의 기준 금리에 대해서 설명해줘? [/INST]

은행은 특정 상품의 가격이나 통화 가치 상승에 자금을 차입해 수익을 창출합니다. 이러한 이자율도 은행으로서는 비용이며 일반적으로 시중 금리에 따른 이자율보다 낮은 수준으로 유지됩니다.

은행의 기준 금리인 경우 보통 시장에서 적용되는 일반적인 이자율보다 낮은 수준으로 유지됩니다. 은행의 일반 은행 업무에 자금을 지원하는 경우 수익성이 비교적 낮은 은행은 일반 이자율보다 높은 금리를 제공해야 합니다.

은행은 보통 다른 금리와 다르게 움직이는 시장 기준 금리와 같은 기준 금리 대신 자체적인 기준을 사용하는 경우가 많으며, 이러한 기준은 일반적으로 시중 은행 금리보다 낮은 수준으로 유지됩니다.

하지만 일반적으로 기준 금리에서 0.05% 또는 0.1%의 '단위'가 존재합니다. 이 기준점은 거래량이 아주 많지 않거나 금융 시장이 혼란에 빠지고 있을 

In [13]:
import locale
def getpreferredencoding(do_setlocale=True):
  return "UTF-8"

locale.getpreferredencoding = getpreferredencoding # 아래 패키지 설치 시 오류 방지

In [12]:
!pip install -q langchain pypdf chromadb sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [32]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline (
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2, # 할루시네이션 방지
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """

### [INST]
instrunction: Answer the question based on your bank knowledge, If you don't know something, say you don't know it

Here is context to help:

{context}

### QUESTION:
{question}


[/INST]
"""

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)



In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
loader = PyPDFLoader("/content/guideline_20230630.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encoding_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name = model_name,
    encode_kwargs = encoding_kwargs
)

db = FAISS.from_documents(texts, hf) # vector store
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k':3}
)



In [36]:
rag_chain = (
    {"context":retriever,"question":RunnablePassthrough()}
    | llm_chain
)

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
result = rag_chain.invoke("자동 납부 동의자료 란?")

for i in result['context']:
  print(f"근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"답변: {result['text']}")


근거: 1 / 8Ⅰ. 자동납부 동의자료란 ?
  □ 자동납부 동의자료란 납부자가 자동납부 신청 시 요금청구기관에 제출하는 출금
동의내역이 포함된 정보를 지칭
  □ 요금청구기관은 납부자로부터 생성한 자동납부 동의자료를 자사의 자동이체서비
스(CMS, 지로, 펌뱅킹) 제공기관을 통해 금융결제원 자동이체 통합관리 시스템으로 
제출
  ※ 자동납부동의자료 제출은 관련 법령*에 의거한 의무사항으로 , 납부자의 자동납부 
신청 시 요금청구기관은 자동납부 등록정보와 자동납부 동의자료를 동시 송신
하여야 함
     * 전자금융거래법 제15조(추심이체의 출금동의 ), 전자금융거래법 시행령 제10조(출금동의의 
방법), 전자금융감독규정 제6조(추심이체 출금 동의의 방법 등)에 의거하며 , 세부사항은 붙
임1(관련 법령) 참조
                             자동납부동의자료 개념도 / 출처: /content/guideline_20230630.pdf - 1 


근거: 2 / 8Ⅱ. 자동납부 동의자료 생성
 1. 자동납부 동의자료 내 정보내역
  □ 요금청구기관은 다음의 납부자 정보내역을 포함한 자동납부 동의자료를 생성
구 분 내 역
필수 정보 요금청구기관명
 납부자 출금은행
 납부자 출금계좌번호
 출금계좌 예금주명
 예금주 생년월일 또는 주민번호대체번호 
 ※법인일 경우 사업자번호
동의방법서면  예금주 날인 또는 서명주)
녹취 ․ ARS  동의음성 또는 ARS 안내음
전자문서  전자서명
선택 정보  자동이체 서비스 종류
신청인과 예금주가 다른 경우 신청인명
 예금주와의 관계
 신청인 날인 또는 서명
    주) 단, 금융거래 제출서류 간소화로 자동납부 동의서류 상의 예금주 날인 또는 서명란이 삭제된 
경우는 예외
※ 요금청구기관은 자동납부 동의와 관련 없는 상기 정보 이외의 개인정보나  
상담내역을 자동납부 동의자료에 포함할 수 없음 / 출처: /content/guideline_20230630.pdf - 2 


근거: 3 / 8 
2. 자동납부 동의방법
